In [41]:
import nltk
import time
from collections import Counter
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV

import csv
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import regexp_tokenize, wordpunct_tokenize,blankline_tokenize
from nltk import PorterStemmer, LancasterStemmer, SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
import re
import string
from collections import Counter
import json
import re as regex
import xgboost as xgb
from sklearn import model_selection, preprocessing
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn.metrics import r2_score


In [79]:
class TwitterData_Initialize():
    data = []
    processed_data = []
    wordlist = []

    featureList = []
    fea_vect=[]
    
    data_model = None
    data_labels = None
    is_testing = False
    
    def initialize(self, csv_file, is_testing_set=False, from_cached=None):
        if from_cached is not None:
            #self.data_model = pd.read_csv(from_cached)
            self.data_model = pd.read_json(from_cached)

            return

        self.is_testing = is_testing_set

        if not is_testing_set:
            #self.data = pd.read_csv(csv_file, header=0, names=["id", "emotion", "text"])
            self.data = pd.read_json(csv_file)

            #self.data = self.data[self.data["emotion"].isin(["positive", "negative", "neutral"])]
        

        self.processed_data = self.data
        self.wordlist = []
        self.data_model = None
        self.data_labels = None
        
    
    
    
    def do_process(self):
        start_time = time.time()
        def stem_and_join(row,stemmer=nltk.PorterStemmer()):
            row["spans"] = list(map(lambda str: stemmer.stem(str.lower()), row["spans"]))
            return row
    
        def tokenize_grams(row):
                #remove numbers
                def remove_numbers(text):
                    return re.sub(r'\d+', '', text)
             
                def to_lower(text):
                    return text.lower()

                #20 Function to remove whitespace
                def remove_whitespace(text):
                    return " ".join(text.split())

                # Function to remove punctuations
                def remove_punctuations(text):
                    text=re.sub(r'[?|*|.|!|+|-]',r'',text)
                    words = nltk.word_tokenize(text)
                    punt_removed = [w for w in words if w.lower() not in string.punctuation]
                    return " ".join(punt_removed)
#30

                # Function to remove stop words
                def remove_stopwords(text, lang='english'):
                        
                    whitelist=["to","on","n't","not","don't","for","up","below","short","long"]

                    stop_words = set(stopwords.words('english'))
                    word_tokens = word_tokenize(text)
                    #filtered_sentence = [w for w in word_tokens if ((not w in stop_words) or (w in whitelist))]
                    filtered_sentence = []
                    for w in word_tokens:
                        if ((w not in stop_words) or (w in whitelist)) :
                            filtered_sentence.append(w)
                
                    ch=" ".join(filtered_sentence)

                    return ch
#40




                # Function to extract n-grams from text
                def get_ngrams(text, n):

                    n_grams = ngrams(nltk.word_tokenize(text), n)
                    list_grams=[ ' '.join(grams) for grams in n_grams]
                    return list_grams


                # Function to apply lemmatization to a list of words
                def words_lemmatizer(text, encoding="utf8"):
                    wordnet_lemmatizer = WordNetLemmatizer()

                    words = nltk.word_tokenize(text)
                    lemma_words = []
                    wl = WordNetLemmatizer()
                    for word in words:
                        pos = find_pos(word)
                        lemma_words.append(wl.lemmatize(word, pos))
                    return " ".join(lemma_words)

                # Function to find part of speech tag for a word
                def find_pos(word):
                    # Part of Speech constants
                    # ADJ, ADJ_SAT, ADV, NOUN, VERB = 'a', 's', 'r', 'n', 'v'
   
                    pos = nltk.pos_tag(nltk.word_tokenize(word))[0][1]
                    # Adjective tags - 'JJ', 'JJR', 'JJS'
                    if pos.lower()[0] == 'j':
                        return 'a'
                    # Adverb tags - 'RB', 'RBR', 'RBS'
                    elif pos.lower()[0] == 'r':
                        return 'r'
                    # Verb tags - 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'
                    elif pos.lower()[0] == 'v':
                        return 'v'
                    # Noun tags - 'NN', 'NNS', 'NNP', 'NNPS'
                    else:
                        return 'n'
    
            
                #convert to string
                idx=row["spans"]
                #ch="".join(x for x in idx if x)
                ch=' '.join(idx)
               
    
                
                
                #words_stemmer(ch, type="PorterStemmer", lang="english", encoding="utf8")
                #Convert to lower case
                txt_num=remove_numbers(ch)

                txt_low=to_lower(txt_num)
                
                txt_ws=remove_whitespace(txt_low)
    

                txt_pun=remove_punctuations(txt_ws)

                txt_final=remove_stopwords(txt_pun)

                texte=words_lemmatizer(txt_final)

                n_grams=get_ngrams(texte, 2)

                row["token_spans"] = n_grams
    
                return row
        #self.processed_data = self.processed_data.apply(stem_and_join, axis=1)
        self.processed_data = self.processed_data.apply(tokenize_grams, axis=1)
        print("--- %s seconds ---" % (time.time() - start_time))

    

    def build_wordlist(self, min_occurrences=0, max_occurences=500, stopwords=nltk.corpus.stopwords.words("english"),
                       ):
        def get_tweets(data):
            start_time = time.time()

            tweets=list(data.processed_data.token_spans)
    
            list_tweet=[]
            for i,chaine in enumerate(tweets):
                ch=" ".join(chaine)
                list_tweet.append(ch)
            print("--- %s seconds ---" % (time.time() - start_time))
    
            return list_tweet
    
        def get_top_n_words(corpus, n=None):
   
            vec = CountVectorizer().fit(corpus)
            bag_of_words = vec.transform(corpus)
            sum_words = bag_of_words.sum(axis=0) 
            words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
            words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
            return words_freq[:n]
        stopwords=[]
        whitelist=[]
        self.wordlist = []
        """
        corpus=get_tweets(self)
        common_words = get_top_n_words(corpus)
        word_df = pd.DataFrame(data={"word": [k for k, v in common_words],
                                     "occurrences": [v for k, v in common_words]},
                               columns=["word", "occurrences"])
        
        
        word_df.to_csv("wordlist_count.csv", index_label="idx")
        self.wordlist = [k for k, v in common_words]
        
        
        """
        words = Counter()
      
        for idx in self.processed_data.index:
            words.update(self.processed_data.loc[idx, "token_spans"])

        for idx, stop_word in enumerate(stopwords):
            if stop_word not in whitelist:
                del words[stop_word]
        

        
                
        print(words.most_common())
        word_df = pd.DataFrame(data={"word": [k for k, v in words.most_common() if min_occurrences < v < max_occurences],
                                     "occurrences": [v for k, v in words.most_common() if min_occurrences < v < max_occurences]},
                               columns=["word", "occurrences"])

        word_df.to_csv("wordlist_copie.csv", index_label="idx")
        self.wordlist = [k for k, v in words.most_common() if min_occurrences < v < max_occurences]
        
        
    def build_data_model(self):
        label_column = []
        Id_column=["ID"]
        label_column = ["label"]

        columns = Id_column + label_column + list(
            map(lambda w: w ,self.wordlist))
        labels = []
        rows = []
        for idx in self.processed_data.index:
            current_row = []

            if True:
                # add label
                current_label = self.processed_data.loc[idx, "sentiment score"]
                current_id = self.processed_data.loc[idx, "id"]
                
                labels.append(current_id)
                labels.append(current_label)
                
                current_row.append(current_id)
                current_row.append(current_label)

            # add bag-of-words
            tokens = set(self.processed_data.loc[idx, "token_spans"])
            for _, word in enumerate(self.wordlist):
                current_row.append(1 if word in tokens else 0)

            rows.append(current_row)

        self.data_model = pd.DataFrame(rows, columns=columns)
        self.data_labels = pd.Series(labels)
        return self.data_model, self.data_labels


In [54]:
class TwitterData_Initialize_test(TwitterData_Initialize):

    
    
    def do_process(self):
        def stem_and_join(row,stemmer=nltk.PorterStemmer()):
            
            row["spans"] = list(map(lambda str: stemmer.stem(str.lower()), row["spans"].split()))
            return row
    
        def tokenize_grams(row):
                #remove numbers
                def remove_numbers(text):
                    return re.sub(r'\d+', '', text)

                def to_lower(text):
                    return text.lower()
                
                #20 Function to remove whitespace
                def remove_whitespace(text):
                    return " ".join(text.split())

                # Function to remove punctuations
                def remove_punctuations(text):
                    text=re.sub(r'[?|*|.|!|+|-]',r'',text)

                    words = nltk.word_tokenize(text)
                    punt_removed = [w for w in words if w.lower() not in string.punctuation]
                    return " ".join(punt_removed)
#30

                # Function to remove stop words
                def remove_stopwords(text, lang='english'):

                    whitelist=["to","for","n't","not","don't","up","below","short","long"]

                    stop_words = set(stopwords.words('english'))
                    word_tokens = word_tokenize(text)
                    #filtered_sentence = [w for w in word_tokens if ((not w in stop_words) or (w in whitelist))]
                    filtered_sentence = []
                    for w in word_tokens:
                        if ((w not in stop_words) or (w in whitelist) ):
                            filtered_sentence.append(w)
                
                    ch=" ".join(filtered_sentence)

                    return ch
#40




                # Function to extract n-grams from text
                def get_ngrams(text, n):

                    n_grams = ngrams(nltk.word_tokenize(text), n)
                    list_grams=[ ' '.join(grams) for grams in n_grams]
                    return list_grams


                # Function to apply lemmatization to a list of words
                def words_lemmatizer(text, encoding="utf8"):
                    wordnet_lemmatizer = WordNetLemmatizer()

                    words = nltk.word_tokenize(text)
                    lemma_words = []
                    wl = WordNetLemmatizer()
                    for word in words:
                        pos = find_pos(word)
                        lemma_words.append(wl.lemmatize(word, pos))
                    return " ".join(lemma_words)

                # Function to find part of speech tag for a word
                def find_pos(word):
                    # Part of Speech constants
                    # ADJ, ADJ_SAT, ADV, NOUN, VERB = 'a', 's', 'r', 'n', 'v'
   
                    pos = nltk.pos_tag(nltk.word_tokenize(word))[0][1]
                    # Adjective tags - 'JJ', 'JJR', 'JJS'
                    if pos.lower()[0] == 'j':
                        return 'a'
                    # Adverb tags - 'RB', 'RBR', 'RBS'
                    elif pos.lower()[0] == 'r':
                        return 'r'
                    # Verb tags - 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'
                    elif pos.lower()[0] == 'v':
                        return 'v'
                    # Noun tags - 'NN', 'NNS', 'NNP', 'NNPS'
                    else:
                        return 'n'
    
            
            
                idx=row["spans"]
                ch=' '.join(idx)


    
                #words_stemmer(ch, type="PorterStemmer", lang="english", encoding="utf8")
                #Convert to lower case
                txt_num=remove_numbers(ch)
                
                
                txt_low=to_lower(txt_num)
                
                txt_ws=remove_whitespace(txt_low)
                    
    
                txt_pun=remove_punctuations(txt_ws)
        
                txt_final=remove_stopwords(txt_pun)

                texte=words_lemmatizer(txt_final)

                n_grams=get_ngrams(texte, 2)
                #print("list ngrams--------------\n",n_grams)

                row["token_spans"] = n_grams
    
                return row
        #self.processed_data = self.processed_data.apply(stem_and_join, axis=1)
        self.processed_data = self.processed_data.apply(tokenize_grams, axis=1)
        
    def build_wordlist(self, min_occurrences=0, max_occurences=500, stopwords=nltk.corpus.stopwords.words("english"),
                      ):
        def get_tweets(data):
            start_time = time.time()

            tweets=list(data.processed_data.token_spans)
    
            list_tweet=[]
            for i,chaine in enumerate(tweets):
                ch=" ".join(chaine)
                list_tweet.append(ch)
            print("--- %s seconds ---" % (time.time() - start_time))
    
            return list_tweet
    
        def get_top_n_words(corpus, n=None):
   
            vec = CountVectorizer().fit(corpus)
            bag_of_words = vec.transform(corpus)
            sum_words = bag_of_words.sum(axis=0) 
            words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
            words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

            return words_freq[:n]

        #whitelist=["to","on","for","up","below","short","long"]
        stopwords=[]
        self.wordlist = []
        """
        corpus=get_tweets(self)
        common_words = get_top_n_words(corpus)
        word_df = pd.DataFrame(data={"word": [k for k, v in common_words],
                                     "occurrences": [v for k, v in common_words]},
                               columns=["word", "occurrences"])
        
        
        word_df.to_csv("wordlist_count_test.csv", index_label="idx")
        self.wordlist = [k for k, v in common_words]
        
        
        
        """

        words = Counter()
      
        for idx in self.processed_data.index:
            words.update(self.processed_data.loc[idx, "token_spans"])

        for idx, stop_word in enumerate(stopwords):
            if stop_word not in whitelist:
                del words[stop_word]
        

        
                
        print(words.most_common())

        word_df = pd.DataFrame(data={"word": [k for k, v in words.most_common() if min_occurrences < v < max_occurences],
                                     "occurrences": [v for k, v in words.most_common() if min_occurrences < v < max_occurences]},
                               columns=["word", "occurrences"])

        word_df.to_csv("wordlist_test.csv", index_label="idx")
        self.wordlist = [k for k, v in words.most_common() if min_occurrences < v < max_occurences]
        
        
    def build_data_model(self):
        
        label_id = ["ID"]

        columns = label_id + list(
            map(lambda w: w ,self.wordlist))
        labels = []
        rows = []
        for idx in self.processed_data.index:
            current_row = []
            if True:
                # add label
                current_id = self.processed_data.loc[idx, "id"]
                
                labels.append(current_id)
                
                current_row.append(current_id)

            

            # add bag-of-words
            tokens = set(self.processed_data.loc[idx, "token_spans"])
            for _, word in enumerate(self.wordlist):
                current_row.append(1 if word in tokens else 0)

            rows.append(current_row)

        self.data_model = pd.DataFrame(rows, columns=columns)
        return self.data_model


In [55]:
data = TwitterData_Initialize()
data.initialize("Microblog_Trainingdata.json")

data.processed_data.head(20)

,cashtag,id,sentiment score,source,spans
0,$FB,719659409228451840,0.366,twitter,[watching for bounce tomorrow]
1,$LUV,719904304207962112,0.638,twitter,[record number of passengers served in 2015]
2,$NFLX,5329774,-0.494,stocktwits,[out $NFLX -.35]
3,$DIA,719891468173844480,0.460,twitter,"[Looking for a strong bounce, Lunchtime rally ..."
4,$PLUG,20091246,0.403,stocktwits,[Very intrigued with the technology and growth...
5,$GMCR,5819749,0.000,stocktwits,"[short worked, puts up]"
6,$IBM,709741154393133056,-0.296,twitter,[overbought]
7,$JOSB,17892972,-0.546,stocktwits,"[absolute garbage still up, stores TOTALLY EMP..."
8,$CSTM,709834259687710720,-0.438,twitter,[Biggest Market Losers]
9,$PYPL,708481442079068160,0.408,twitter,[Love this company long time.]


In [56]:
data_test = TwitterData_Initialize_test()
data_test.initialize("Microblog_Trialdata.json")

data_test.processed_data.head()
test_data=data_test.processed_data.copy()

In [57]:
data.do_process()

C:\Users\Globalnet\Anaconda3\lib\site-packages\ipykernel_launcher.py:88: DeprecationWarning: generator 'ngrams' raised StopIteration


--- 229.51958346366882 seconds ---


In [58]:
data.processed_data.head(20)

,cashtag,id,sentiment score,source,spans,token_spans
0,$FB,719659409228451840,0.366,twitter,[watching for bounce tomorrow],"[watch for, for bounce, bounce tomorrow]"
1,$LUV,719904304207962112,0.638,twitter,[record number of passengers served in 2015],"[record number, number passenger, passenger se..."
2,$NFLX,5329774,-0.494,stocktwits,[out $NFLX -.35],[]
3,$DIA,719891468173844480,0.460,twitter,"[Looking for a strong bounce, Lunchtime rally ...","[look for, for strong, strong bounce, bounce l..."
4,$PLUG,20091246,0.403,stocktwits,[Very intrigued with the technology and growth...,"[intrigue technology, technology growth, growt..."
5,$GMCR,5819749,0.000,stocktwits,"[short worked, puts up]","[short work, work put, put up]"
6,$IBM,709741154393133056,-0.296,twitter,[overbought],[]
7,$JOSB,17892972,-0.546,stocktwits,"[absolute garbage still up, stores TOTALLY EMP...","[absolute garbage, garbage still, still up, up..."
8,$CSTM,709834259687710720,-0.438,twitter,[Biggest Market Losers],"[big market, market loser]"
9,$PYPL,708481442079068160,0.408,twitter,[Love this company long time.],"[love company, company long, long time]"


In [80]:

data.build_wordlist()
#data.wordlist.sort()

print(len(data.wordlist))

[('unusual call', 16), ('look like', 15), ('stochastic overbought', 13), ('ready to', 13), ('call activity', 12), ("today 's", 12), ('top stock', 12), ('still hold', 12), ('look for', 11), ('to break', 11), ('nice day', 11), ('day rally', 11), ('new high', 10), ('start to', 10), ('to buy', 10), ('short setup', 9), ('time to', 9), ('sector stock', 9), ('stock lead', 9), ('lead today', 9), ('up for', 9), ('big market', 8), ('market loser', 8), ('setup look', 8), ('look nicereally', 8), ('nicereally nice', 8), ('add to', 8), ('next week', 8), ('look good', 8), ('on cuba', 8), ('go to', 8), ('want to', 7), ('to go', 7), ('to sell', 7), ('move upper', 7), ('upper bollinger', 7), ('bollinger band', 7), ('high call', 7), ('call to', 7), ('to put', 7), ('put ratio', 7), ('buy dip', 7), ('hold up', 7), ('good to', 7), ('load up', 7), ('up on', 7), ('try to', 7), ('insider sell', 6), ('set up', 6), ('pie sky', 6), ('sell rating', 6), ('gap fill', 6), ('move up', 6), ('make new', 6), ('high after

In [64]:
bow, labels = data.build_data_model()
bow.head(20)


,ID,label,'auto stock,'best ever,'d short,'highly cover,'large player,'let go,'ll see,'ll take,...,yield announce,zigzag reversal,ziv lose,znga copyright,zombie stock,zone bot,zone strong,zone today,zuck told,zuckerberg amaze
0,719659409228451840,0.366,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,719904304207962112,0.638,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5329774,-0.494,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,719891468173844480,0.460,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,20091246,0.403,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5819749,0.000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,709741154393133056,-0.296,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,17892972,-0.546,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,709834259687710720,-0.438,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,708481442079068160,0.408,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
len(bow.columns)

4088

In [67]:
data_test.do_process()
data_test.processed_data.head(20)
data_test.build_wordlist()
data_test.wordlist.sort()

print(len(data_test.wordlist))
bow_test= data_test.build_data_model()
bow_test.head(5)


[('put little', 2), ('little f', 2), ('f short', 2), ('long position', 2), ('time to', 2), ('buying opportunity', 1), ('scale up', 1), ('up long', 1), ('to sell', 1), ('sell bank', 1), ('enter long', 1), ('picked up', 1), ('to accumulate', 1), ('accumulate for', 1), ('for long', 1), ('position far', 1), ('far upside', 1), ('upside downside', 1), ('look for', 1), ('for strong', 1), ('strong bounce', 1), ('bounce lunchtime', 1), ('lunchtime rally', 1), ('rally come', 1), ('intrigue technology', 1), ('technology growth', 1), ('growth potential', 1), ('short work', 1), ('work put', 1), ('put up', 1), ('big market', 1), ('market loser', 1), ('goog googl', 1), ('googl would', 1), ('would suck', 1), ('buying sbux', 1), ('sbux dip', 1), ('short below', 1), ('below overbought', 1), ("n't put", 1)]
40


,ID,accumulate for,below overbought,big market,bounce lunchtime,buying opportunity,buying sbux,enter long,f short,far upside,...,short work,strong bounce,technology growth,time to,to accumulate,to sell,up long,upside downside,work put,would suck
0,5540055,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,10752226,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10920221,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,12971398,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,16142438,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0


In [68]:
len(bow_test.columns)


41

In [69]:
for word in data.wordlist:
    current_row=[]
    if (not word  in data_test.wordlist):
        # add label
        for idx in bow_test.index:
            current_row.append(0)
        bow_test[word]=current_row

    
bow_test.head()


,ID,accumulate for,below overbought,big market,bounce lunchtime,buying opportunity,buying sbux,enter long,f short,far upside,...,yield announce,zigzag reversal,ziv lose,znga copyright,zombie stock,zone bot,zone strong,zone today,zuck told,zuckerberg amaze
0,5540055,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,10752226,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10920221,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,12971398,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,16142438,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
len(bow_test.columns)


4099

In [71]:
#bow_test.sort_index(axis = 1, ascending = False)
#bow.sort_index(axis = 1, ascending = True)


In [72]:
#convert bow dataframe to numpy matrix

y_train=bow["label"]
X_train=bow.drop({"ID","label"},axis=1)

In [73]:
#Fitting XGB regressor with parameters obtained by Grid searchCV
params={'colsample_bytree': 1.0,
 'gamma': 0.3,
 'max_depth': 4,
 'min_child_weight': 4,
 'subsample': 0.8}
model = xgb.XGBRegressor(max_depth=4, min_child_weight= 4,gamma=0.3,subsample=0.8,colsample_bytree=1.0)

model.fit(X_train,y_train)
print (model)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1.0, gamma=0.3, learning_rate=0.1,
       max_delta_step=0, max_depth=4, min_child_weight=4, missing=None,
       n_estimators=100, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=0.8)


In [74]:
"""#get tweets contained in test data after train-test_split
tweet_row=[]
labels=["ID","tweet"]
for num in id_vals:
    current_row=[]
    current_id = bow.loc[num, "ID"]
                
    current_tweet=data.processed_data.loc[data.processed_data['id'] == current_id,"spans"].iloc[0]

                
    current_row.append(current_id)
    current_row.append(current_tweet)
    

   

    tweet_row.append(current_row) 
tweet_row"""

'#get tweets contained in test data after train-test_split\ntweet_row=[]\nlabels=["ID","tweet"]\nfor num in id_vals:\n    current_row=[]\n    current_id = bow.loc[num, "ID"]\n                \n    current_tweet=data.processed_data.loc[data.processed_data[\'id\'] == current_id,"spans"].iloc[0]\n\n                \n    current_row.append(current_id)\n    current_row.append(current_tweet)\n    \n\n   \n\n    tweet_row.append(current_row) \ntweet_row'

In [75]:
#reorganize test columns as the train columns

#X_test = bow_test.as_matrix()

#X_test

bow_test=bow_test[bow.drop(["label"],axis=1).columns]
print(len(bow_test.columns))
X_test=bow_test.drop({"ID"},axis=1)
y_test=data_test.processed_data["sentiment score"]
id_vals=X_test.index

4087


In [76]:
#Predict 
#output=grid.best_estimator_.predict(X_test)
tweet_row=list(data_test.processed_data["spans"])
old_pred=list(data_test.processed_data["sentiment score"])
output = model.predict(X_test)
#final_df = pd.DataFrame(tweet_row, columns=labels)
final_df = pd.DataFrame()
final_df["tweet"] = tweet_row

final_df["old_pred"] = old_pred

final_df["Prediction"] = output
final_df.to_csv("Output_1.csv",sep=",")
final_df.head(20)

,tweet,old_pred,Prediction
0,[Putting on a little $F short],-0.454,0.115184
1,[short some],-0.464,0.115184
2,[buying opportunity],0.445,0.115184
3,[Scaling Up on Long Position],0.661,0.115184
4,[its time to sell banks],-0.763,-0.388433
5,[Entering long],0.627,0.115184
6,[picked some up],0.653,0.115184
7,"[time to accumulate for a long position, far m...",0.668,-0.057362
8,"[Looking for a strong bounce, Lunchtime rally ...",0.460,0.222146
9,[Very intrigued with the technology and growth...,0.403,0.115184


In [78]:
#test model accuracy
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


print("mean squared error:" ,mean_squared_error(output, y_test))
print("R2 score:" ,r2_score(output,y_test))

mean squared error: 0.22173702040500798
R2 score: -8.45936072661542
